In [1]:
import pandas as pd
import plotly
import plotly.express as px

nhs_colours = ['#00A499', '#FFB81C']

# Appointments Analysis

### What type of appointment did you get?

In [2]:
# import csv file
appointment_types = pd.read_csv('data/Appoinment_types.csv', sep=',', header=0)
# rename blank first col
appointment_types.rename({' ':'Appointemnt Types'}, axis=1, inplace=True)

# create a dict of appointments and there coresponding type
grouptype = {
    'To speak to someone on the phone': 'Remote',
    'To speak to someone online (for example on a video call)': 'Remote',
    'To see someone at another general practice location': 'Face-to-face',
    'To see someone at my GP practice': 'Face-to-face',
    'For a home visit': 'Face-to-face'
    }

# make a new col that maps the groups to the appointments
appointment_types['Group Type'] = appointment_types['Appointemnt Types'].map(grouptype)

# now we can groupby type and sum the yearly counts
appointment_types_group = appointment_types.groupby('Group Type').sum().reset_index()
# melt is like an inverse pivot, changing the data to 'long' format (one observeation per row)
appointment_types_group = pd.melt(
    appointment_types_group,
    id_vars='Group Type',
    value_vars=[
        'Q1 2018',
        'Q1 2019',
        'Q1 2020',
        'Q1 2021'
        ]
    )
appointment_types_group.columns = ["Appointment Type", "Year", "Count"]

# use groupby on years to get the total counts
appointment_year_total = appointment_types_group.groupby('Year').sum().reset_index()
appointment_year_total.rename(columns={'Count': 'Total'}, inplace=True)

# merge the totals back to the df
appointment_types_group = pd.merge(appointment_types_group, appointment_year_total, on='Year', how='left')
# calculate percentages
appointment_types_group['Percent'] = appointment_types_group['Count']/appointment_types_group['Total']
appointment_types_group

,Appointment Type,Year,Count,Total,Percent
0,Face-to-face,Q1 2018,574310,633357,0.906771
1,Remote,Q1 2018,59047,633357,0.093229
2,Face-to-face,Q1 2019,579817,641169,0.904312
3,Remote,Q1 2019,61352,641169,0.095688
4,Face-to-face,Q1 2020,545147,605492,0.900337
5,Remote,Q1 2020,60345,605492,0.099663
6,Face-to-face,Q1 2021,333309,659667,0.505269
7,Remote,Q1 2021,326358,659667,0.494731


### Appointments: data visualisation

In [3]:
# plot the long (tidy) dataframe
fig = px.bar(
    appointment_types_group,
    x="Year",
    y="Percent",
    color="Appointment Type",
    color_discrete_sequence=nhs_colours,
    barmode="stack",
    text="Percent",
    height=350,
)

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    yaxis_title='Of survey respondents (%)',
    yaxis=dict(tickformat = "%"),
    xaxis_title=None,
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=0,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    hovermode="x unified",
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)

# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{fullData.name}</b>: %{text:.1%}<extra></extra>',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    texttemplate='%{text:.1%}',
    textposition='inside'
)
fig.update_xaxes(type='category')
fig.show()

In [4]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/remote_appts.html", "w") as file:
    file.write(plotly_obj)

# GP online services

### Which of the following general practice online services have you used in the past 12 months?

In [5]:
online_gp_services = pd.read_csv (r'data/Online_gp_services.csv', sep=',', header=0).dropna(axis='columns')
online_gp_services = online_gp_services.rename(columns={
    "75_Over_P": "Aged 75+",
    '16_24_P': "Aged 16-24",
    'National_P': "All Patients"})
online_gp_services = pd.melt(
    online_gp_services,
    id_vars=['Activity', 'Lable'],
    value_vars=[
        "Aged 75+",
        "Aged 16-24",
        "All Patients"])
online_gp_services.columns = ["Online Activity", "Lable", "Age Group", "Percent"]

online_gp_services_75 = online_gp_services[~online_gp_services['Online Activity'].str.contains("None of these")]
online_gp_services_75 = online_gp_services_75[~online_gp_services_75['Age Group'].str.contains("Aged 16-24")]
online_gp_services_75 = online_gp_services_75.sort_values(['Percent', "Age Group"], ascending=False)
online_gp_services_75

,Online Activity,Lable,Age Group,Percent
11,Ordering repeat prescriptions online,Order repeat<br>prescription,All Patients,0.26
1,Ordering repeat prescriptions online,Order repeat<br>prescription,Aged 75+,0.24
10,Booking appointments online,Book an<br>appointment,All Patients,0.19
13,Had an online consultation or appointment,Had a Consultation<br>or appointment,All Patients,0.18
0,Booking appointments online,Book an<br>appointment,Aged 75+,0.11
3,Had an online consultation or appointment,Had a Consultation<br>or appointment,Aged 75+,0.11
12,Accessing my medical records online,Access medical<br>record,All Patients,0.07
2,Accessing my medical records online,Access medical<br>record,Aged 75+,0.03


In [6]:

# plot the long (tidy) dataframe
fig = px.bar(
    online_gp_services_75,
    x="Online Activity",
    y="Percent",
    color="Age Group",
    color_discrete_sequence=nhs_colours,
    barmode="group",
    text="Percent",
    height=350,
)

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0,0,0, 0)", "paper_bgcolor": "rgba(0,0,0, 0)"},
    yaxis_title='Of survey respondents (%)',
    yaxis=dict(
        tickformat = "%",
        range=[0, 0.3]),
    xaxis=dict(
        tickmode = 'array',
        tickvals = online_gp_services_75['Online Activity'].tolist(),
        ticktext = online_gp_services_75['Lable'].tolist(),
        #tickangle = 0
    ),
    xaxis_title=None,
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=0,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    hovermode="x unified",
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)
# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{fullData.name}</b>: %{text:.1%}<extra></extra>',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    texttemplate='%{text:.1%}',
    textposition='inside'
    )
fig.update_xaxes(type='category')
fig.show()

In [7]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/online_activities_75.html", "w") as file:
    file.write(plotly_obj)

In [8]:
online_gp_services = pd.read_csv (r'data/Online_gp_services.csv', sep=',', header=0).dropna(axis='columns')
online_gp_services = online_gp_services.rename(columns={
    "75_Over_P": "Aged 75+",
    '16_24_P': "Aged 16-24",
    'National_P': "All Patients"})
online_gp_services = pd.melt(
    online_gp_services,
    id_vars=['Activity', 'Lable'],
    value_vars=[
        "Aged 75+",
        "Aged 16-24",
        "All Patients"])
online_gp_services.columns = ["Online Activity", "Lable", "Age Group", "Percent"]

online_gp_services_25 = online_gp_services[~online_gp_services['Online Activity'].str.contains("None of these")]
online_gp_services_25 = online_gp_services_25[~online_gp_services_25['Age Group'].str.contains("Aged 75+")]
online_gp_services_25 = online_gp_services_25.sort_values(['Percent', "Age Group"], ascending=False)
online_gp_services_25

,Online Activity,Lable,Age Group,Percent
11,Ordering repeat prescriptions online,Order repeat<br>prescription,All Patients,0.26
10,Booking appointments online,Book an<br>appointment,All Patients,0.19
5,Booking appointments online,Book an<br>appointment,Aged 16-24,0.19
8,Had an online consultation or appointment,Had a Consultation<br>or appointment,Aged 16-24,0.19
13,Had an online consultation or appointment,Had a Consultation<br>or appointment,All Patients,0.18
6,Ordering repeat prescriptions online,Order repeat<br>prescription,Aged 16-24,0.17
12,Accessing my medical records online,Access medical<br>record,All Patients,0.07
7,Accessing my medical records online,Access medical<br>record,Aged 16-24,0.07


In [9]:

# plot the long (tidy) dataframe
fig = px.bar(
    online_gp_services_25,
    x="Online Activity",
    y="Percent",
    color="Age Group",
    color_discrete_sequence=nhs_colours,
    barmode="group",
    text="Percent",
    height=350,
)

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0,0,0, 0)", "paper_bgcolor": "rgba(0,0,0, 0)"},
    yaxis_title='Of survey respondents (%)',
    yaxis=dict(
        tickformat = "%",
        range=[0, 0.3]),
    xaxis=dict(
        tickmode = 'array',
        tickvals = online_gp_services_25['Online Activity'].tolist(),
        ticktext = online_gp_services_25['Lable'].tolist(),
        #tickangle = 0
    ),
    xaxis_title=None,
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=0,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    hovermode="x unified",
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)
# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{fullData.name}</b>: %{text:.1%}<extra></extra>',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    texttemplate='%{text:.1%}',
    textposition='inside'
    )
fig.update_xaxes(type='category')
fig.show()

In [10]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/online_activities_25.html", "w") as file:
    file.write(plotly_obj)

## Trends - Using GP Websites

## How easy is it to use your GP practice’s website to look for information or access services?

In [11]:
# Over 75s (2018 - 2021)
ease_of_use_75 = pd.read_csv (r'data/Ease_of_use_over_time_75+.csv', header = 0)
ease_of_use_75.columns = ["Year", "Percent"]

# 16-24s (2018 - 2021)
ease_16_24 = pd.read_csv (r'data/use_16_24.csv', header = 0)                                                                                                                                                 
ease_16_24.columns = ["Year", "Age 16-24"]

# Long term health conditions (2018 - 2021)
trend_ease_use_LTC = pd.read_csv (r'data/trend_ease_use_2021_time_long_term_condition.csv')
trend_ease_use_LTC = trend_ease_use_LTC.dropna(axis='rows')
trend_ease_use_LTC.columns = ["Year", "Long-term Cond."]

ease_of_use_75['Age Group'] = 'Age 75+'
ease_of_use_75['Difficult'] = 1-ease_of_use_75['Percent']
ease_of_use_75

,Year,Percent,Age Group,Difficult
0,Q1 2018,0.84,Age 75+,0.16
1,Q1 2019,0.81,Age 75+,0.19
2,Q1 2020,0.80,Age 75+,0.20
3,Q1 2021,0.75,Age 75+,0.25


In [12]:

# plot the long (tidy) dataframe
fig = px.bar(
    ease_of_use_75,
    x="Year",
    y="Difficult",
    color="Age Group",
    color_discrete_sequence=nhs_colours,
    barmode="group",
    text="Difficult",
    height=350,
)

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    yaxis_title='Of survey respondents (%)',
    xaxis_title=None,
    yaxis=dict(
        tickformat = "%",
        range=[0, 0.3]),
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=0,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    hovermode="x unified",
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)
# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{fullData.name}</b>: %{text:.0%}<extra></extra>',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    texttemplate='%{text:.0%}',
    textposition='inside',
    textangle = 0
    )
fig.update_xaxes(type='category')
fig.show()

In [13]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/ease_of_use.html", "w") as file:
    file.write(plotly_obj)

##  Which of the following general practice online services have you used in the past 12 months

In [14]:
# Q3. Which of the following general practice online services have you used in the past 12 months?
online_gp_services_LTC = pd.read_csv (r'data/GPPS_Long term health conditions 2021.csv', header = 0)
online_gp_services_LTC


,Activity,Lable,Condition,Count of Respondents,Total Respondents
0,Booking appointments online,Book an<br>appointment,Long-term condition,78948,402294
1,Ordering repeat prescriptions online,Order repeat<br>prescription,Long-term condition,148279,402294
2,Accessing my medical records online,Access medical<br>record,Long-term condition,35153,402294
3,Had an online consultation or appointment,Had a consultation<br>or appointment,Long-term condition,76365,402294
4,None of these,None of these,Long-term condition,196402,402294
5,Booking appointments online,Book an<br>appointment,No long-term condition,62795,343782
6,Ordering repeat prescriptions online,Order repeat<br>prescription,No long-term condition,50856,343782
7,Accessing my medical records online,Access medical<br>record,No long-term condition,19581,343782
8,Had an online consultation or appointment,Had a consultation<br>or appointment,No long-term condition,59085,343782
9,None of these,None of these,No long-term condition,213146,343782


In [15]:
# calculate percentages
online_gp_services_LTC['Percent'] = online_gp_services_LTC['Count of Respondents']/online_gp_services_LTC['Total Respondents']
online_gp_services_LTC = online_gp_services_LTC.sort_values(['Percent', 'Condition'], ascending=False)
online_gp_services_LTC = online_gp_services_LTC[~online_gp_services_LTC['Activity'].str.contains("None of these")]
online_gp_services_LTC

,Activity,Lable,Condition,Count of Respondents,Total Respondents,Percent
1,Ordering repeat prescriptions online,Order repeat<br>prescription,Long-term condition,148279,402294,0.368584
0,Booking appointments online,Book an<br>appointment,Long-term condition,78948,402294,0.196245
3,Had an online consultation or appointment,Had a consultation<br>or appointment,Long-term condition,76365,402294,0.189824
5,Booking appointments online,Book an<br>appointment,No long-term condition,62795,343782,0.182659
8,Had an online consultation or appointment,Had a consultation<br>or appointment,No long-term condition,59085,343782,0.171868
6,Ordering repeat prescriptions online,Order repeat<br>prescription,No long-term condition,50856,343782,0.147931
2,Accessing my medical records online,Access medical<br>record,Long-term condition,35153,402294,0.087381
7,Accessing my medical records online,Access medical<br>record,No long-term condition,19581,343782,0.056958


In [16]:
# plot the long (tidy) dataframe
fig = px.bar(
    online_gp_services_LTC,
    x="Activity",
    y="Percent",
    color="Condition",
    color_discrete_sequence=nhs_colours,
    barmode="group",
    text="Percent",
    height=350,
)

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0,0,0, 0)", "paper_bgcolor": "rgba(0,0,0, 0)"},
    yaxis_title='Of survey respondents (%)',
    xaxis=dict(
        tickmode = 'array',
        tickvals = online_gp_services_LTC['Activity'].tolist(),
        ticktext = online_gp_services_LTC['Lable'].tolist(),
        #tickangle = 45
    ),
    yaxis=dict(
        tickformat = "%",
        range=[0, 0.4]
    ),
    xaxis_title=None,
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=0,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    hovermode="x unified",
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)
# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{fullData.name}</b>: %{text:.1%}<extra></extra>',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    texttemplate='%{text:.1%}',
    textposition='inside',
    #textangle = 0
    )
fig.update_xaxes(type='category')
fig.show()

In [17]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/online_activities_ltc.html", "w") as file:
    file.write(plotly_obj)

## Ease of Use (new)

In [2]:
1# import ecxel sheet
df = pd.read_excel(open('data/GPPS_ease_of_use.xls','rb'), sheet_name='All Results')
df = df[['Result','Group','2018 (%)','2019 (%)','2020 (%)','2021 (%)']]
df = df[~df['Result'].str.contains("Total")]
df = df[~df['Result'].str.contains("Easy")]
df.rename({
    '2018 (%)':'Q1 2018',
    '2019 (%)':'Q1 2019',
    '2020 (%)':'Q1 2020',
    '2021 (%)':'Q1 2021',
    }, axis=1, inplace=True)


df_group = pd.melt(
    df,
    id_vars=['Group'],
    value_vars=[
        'Q1 2018',
        'Q1 2019',
        'Q1 2020',
        'Q1 2021'
        ]
    )
df_group.columns = ["Group", "Year", "Percent"]
df_group = df_group.sort_values(['Year','Percent'], ascending=True)
df_group = df_group[~df_group['Group'].str.contains("Hearing and/or sight loss")]
df_group


,Group,Year,Percent
0,All Patients,Q1 2018,0.22
1,Learning disability,Q1 2018,0.27
3,All Patients,Q1 2019,0.23
4,Learning disability,Q1 2019,0.30
6,All Patients,Q1 2020,0.24
7,Learning disability,Q1 2020,0.30
9,All Patients,Q1 2021,0.25
10,Learning disability,Q1 2021,0.31


In [3]:

# plot the long (tidy) dataframe
fig = px.bar(
    df_group,
    x="Year",
    y="Percent",
    color="Group",
    color_discrete_sequence=nhs_colours,
    barmode="group",
    text="Percent",
    height=350,
)

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0,0,0, 0)", "paper_bgcolor": "rgba(0,0,0, 0)"},
    yaxis_title='GP practice website<br> not easy to use (%)',
    yaxis=dict(
        tickformat = "%",
        range=[0, 0.4]),
    xaxis_title=None,
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=0,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    hovermode="x unified",
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)
# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{fullData.name}</b>: %{text:.1%}<extra></extra>',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    texttemplate='%{text:.1%}',
    textposition='inside'
    )
fig.update_xaxes(type='category')
fig.show()

In [4]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/ease_of_use_new.html", "w") as file:
    file.write(plotly_obj)